# pygameを試す
pygameは、現時点でPython 3.5の環境じゃないとインストールが出来ないので、仮想環境を作ってpygameを試す。
  - pygameが動くpython 3.5の仮想環境を構築する。 `>conda create -n py35 python=3.5`
  - pygameをインストールする。 `>conda install -c cogsci pygame`
  - プログラムを試す。

## 1. python3.5の仮想環境構築
### 1-1. 現在の環境を確認
baseの環境のみが存在している。
~~~
(base) C:\***>conda info -e
# conda environments:
base                  *  C:\***\anaconda3
~~~

### 1-2. py35という名前で仮想環境を作成
現時点でpygameがpython3.5に対応しているため、python3.5をインストールする。
~~~
(base) C:\***>conda create -n py35 python=3.5

~~~

### 1-3. 仮想環境が出来たか確認
py35という名前の仮想環境が出来ている。\*が付いた方がactiveな環境。
~~~
(base) C:\***>conda info -e
# conda environments:
base                  *  C:\***\anaconda3
py35                     C:\***\anaconda3\envs\py35
~~~

### 1-4. 仮想環境を切り替え
py35をactiveにする。
~~~
(base) C:\***>activate py35
base                     C:\***\anaconda3
py35                  *  C:\***\anaconda3\envs\py35
~~~

### 1-5. py35環境にJupyterとSpyderをインストールする
~~~
(py35) C:\***>conda install jupyter
(py35) C:\***>conda install spyder
~~~

### 1-6. 環境をアップデートしないとうまく動かなかったので、アップデートする。

~~~
>conda update –-all
~~~

### py35の仮想環境を削除する場合。
一旦対象をdeactivate。（baseをacctivate）し、ターミナルを再起動する。その後、py35の削除処理を行う。
~~~
(py35) C:\***>conda.bat activate base
(base) C:\***>conda remove -n py35 --all
~~~

## 2. プログラムを試す

In [1]:
import pygame
from pygame.locals import MOUSEBUTTONDOWN, QUIT
import sys
import random

In [2]:
class Agent:
    def __init__(self, x, y, vx, vy):
        self.x, self.y = x, y # position 
        self.vx, self.vy = vx, vy # velocity
        
    def update(self, WINDOW_W, WINDOW_H):
        self.x += self.vx
        self.y += self.vy
        
        #Right edge.
        if self.x > WINDOW_W:
            self.vx, self.x = -self.vx, WINDOW_W
            return
        
        #Left edge.
        if self.x < 0:
            self.vx, self.x = -self.vx, 0
            return
        
        #Upper edge.
        if self.y > WINDOW_H:
            self.vy, self.y = -self.vy, WINDOW_H
            return
        #Lower edge.
        
        if self.y < 0:
            self.vy, self.y = -self.vy, 0
            return

    def draw(self, screen):
        x, y = int(self.x), int(self.y)
        pygame.draw.circle(screen, (255,0,0),(x,y),5)

In [3]:
#main
WINDOW_W, WINDOW_H = 1000, 700
pygame.init()
#Create window.
screen = pygame.display.set_mode((WINDOW_W, WINDOW_H))
clock = pygame.time.Clock()
font = pygame.font.Font(None, 28)
agent_list = []

while True:
    clock.tick(60)
    
    # Fill screen with white.
    screen.fill((255, 255, 255))
    
    # Process each agent.
    for a in agent_list:
        a.update(WINDOW_W, WINDOW_H)
        a.draw(screen)
    
    # Show the number of the agent.
    sl = "agents : " + str(len(agent_list))
    text = font.render(sl, True, (0,0,0))
    screen.blit(text, (1, 1))

    # Process events.
    for e in pygame.event.get():
        # When clicked the left button on mouse.
        if e.type == MOUSEBUTTONDOWN and e.button==1:
            # Set velocity by use a random number.
            vx = random.uniform(-1.5,1.5)
            vy = random.uniform(-1.5,1.5)
            # create agents.
            a = Agent(e.pos[0], e.pos[1], vx, vy)
            # Add created agent into the agent list.
            agent_list.append(a)
        if e.type == QUIT:
            pygame.quit()
            sys.exit()
        
    pygame.display.update()
    

SystemExit: 

C:\Users\10007434\AppData\Local\Continuum\anaconda3\envs\py35\lib\site-packages\IPython\core\interactiveshell.py:2969: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
